The objective is to create a dashboard using plotly library dash.plotly.com that contains one chart and one filter.<br>
<br>
Use this dataset https://www.kaggle.com/carrie1/ecommerce-data<br>
Build a cohort analysis chart (something similar to whatever in this blog https://clevertap.com/blog/cohort-analysis/)<br>
Add a country filter to the dashboard<br>
Deploy the app and make it accessible<br>
<br>
Bonus (you don’t have to do it):<br>
Dockerize your application<br>
Use https for your deployment (you can use let’s encrypt)<br>

In [272]:
import pandas as pd 
import datetime as dt 

In [273]:
data = pd.read_csv('data/data.csv', delimiter=',', encoding = "ISO-8859-1")

In [274]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Data cleaning

In [275]:
data.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

Drop unnecessary columns and null values

In [276]:
data = data.drop(['InvoiceNo', 'StockCode', 'Description'], axis=1)
data = data.dropna() 

In [277]:
print("Range Quantity \t\t:", data.Quantity.min(), " to ", data.Quantity.max()) 
print("Range UnitPrice \t:", data.UnitPrice.min(), " to ", data.UnitPrice.max()) 

Range Quantity 		: -80995  to  80995
Range UnitPrice 	: 0.0  to  38970.0


Remove negative value in Quantity, UnitPrice columns

In [278]:
data = data[data.Quantity > 0] 
data = data[data.UnitPrice > 0]

In [279]:
data.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [280]:
data.isnull().sum().sort_values(ascending=False)

Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

convert InvoiceDate to Date data type

In [281]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format='%m/%d/%Y %H:%M')

In [282]:
# remove the time from InvoiceDate
data['InvoiceDate'] = data['InvoiceDate'].dt.date

In [283]:
cleaned_data = data.copy()
cleaned_data.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6,2010-12-01,2.55,17850.0,United Kingdom
1,6,2010-12-01,3.39,17850.0,United Kingdom
2,8,2010-12-01,2.75,17850.0,United Kingdom
3,6,2010-12-01,3.39,17850.0,United Kingdom
4,6,2010-12-01,3.39,17850.0,United Kingdom


# Cohort Analysis

In [284]:
# Create the Cohort
def get_month(x): return dt.datetime(x.year, x.month, 1)
cleaned_data['InvoiceMonth'] = cleaned_data['InvoiceDate'].apply(get_month)# Create InvoiceMonth column
grouping = cleaned_data.groupby('CustomerID')['InvoiceMonth']# Group by CustomerID and select the InvoiceDay value
cleaned_data['CohortMonth'] = grouping.transform('min')# Assign a minimum InvoiceDay value to the dataset

In [285]:
#Extract integer values from data
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [286]:
invoice_year, invoice_month, _ = get_date_int(cleaned_data, 'InvoiceMonth')# Get the integers for date parts from the `InvoiceMonth` column
cohort_year, cohort_month, _ = get_date_int(cleaned_data, 'CohortMonth')# Get the integers for date parts from the `CohortMonth` column

In [287]:
years_diff = invoice_year - cohort_year # Calculate difference in years
months_diff = invoice_month - cohort_month # Calculate difference in months

In [288]:
cleaned_data['CohortIndex'] = years_diff * 12 + months_diff + 1 # Extract the difference in months from all previous values

# Dash

In [289]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [290]:
app = Dash(__name__)

In [291]:
server = app.server

In [292]:
countries = cleaned_data['Country'].unique()

In [293]:
# App layout
app.layout = html.Div([
    html.H1("cohort analysis Dashboards with Dash", style={'text-align': 'center'}),
    dcc.Dropdown(
        id="slct_country",
        options=[{'label': i, 'value': i} for i in countries],
        multi=False,
        value=countries[0],
        style={'width': "40%", 'align': 'center'}
        ),
    dcc.Graph(id='my_heatmap'),
])

In [294]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    Output('my_heatmap', 'figure'),
    Input('slct_country', 'value')
)

def update_graph(option_slctd):
    country_choosen_data = cleaned_data[cleaned_data['Country'] == option_slctd]
    
    ## count monthly active customers from each cohort
    grouping = country_choosen_data.groupby(['CohortMonth', 'CohortIndex'])
    cohort_data = grouping['CustomerID'].apply(pd.Series.nunique)#Count the number of unique values per customer ID
    cohort_data = cohort_data.reset_index()
    cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

    # Retention Rate
    cohort_sizes = cohort_counts.iloc[:,0]#divide each column by value of the first(cohort sizes) to find retention rate
    retention = cohort_counts.divide(cohort_sizes, axis=0)
    retention.round(2) * 100
    
    fig = px.imshow(retention, text_auto='.0%', color_continuous_scale='Blues', zmax=0.5, zmin=0.0)

    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2022 15:41:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2022 15:41:22] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jun/2022 15:41:22] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jun/2022 15:41:22] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jun/2022 15:41:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2022 15:41:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2022 15:41:23] "POST /_dash-update-component HTTP/1.1" 200 -
